# DS4DS Homework Exercise Sheet 10

**General Instructions:**

- Collaborations between students during problem-solving phase on a discussion basis is OK
- However: individual code programming and submissions per student are required
- Code sharing is strictly prohibited
- We will run checks for shared code, general plagiarism and AI-generated solutions
- Any fraud attempt will lead to an auto fail of the entire course
- Do not use any additional packages except for those provided in the task templates
- Please use Julia Version 1.10.x to ensure compatibility
- Please only write between the `#--- YOUR CODE STARTS HERE ---#` and `#--- YOUR CODE ENDS HERE ---#` comments
- Please do not delete, add any cells or overwrite cells other than the solution cells (**Tip:** If you use a jupyerhub IDE, you should not be able to add or delete cells and write in the non-solution cells by default)

In [1]:
using OrdinaryDiffEq
using LinearAlgebra
using Optim
using LaTeXStrings
using MAT

## Task 1: Parameter identification of the double compound pendulum - (5 points)

The goal of this exercise is to perform parameter identification for a non-linear dynamic system. As a system, we consider the double component pendulum ([Source](https://en.wikipedia.org/wiki/Double_pendulum)), which is shown in the following figure.

<img src="./double_pendulum.png" alt="Double pendulum image" width="600"/>

This system consists of two pendulums connected in series, each with its own mass and length (in contrast to the mathematical pendulum where only point masses at the pole tips were assumed!).

In order to obtain the equations of motion and thus the necessary ODEs, we want to utilize Lagragian mechanics ([Source](https://en.wikipedia.org/wiki/Lagrangian_mechanics)). Unlike Newtonian mechanics, which relies on forces and accelerations, Lagrangian mechanics focuses on the concept of energy and utilizes a mathematical function called the Lagrangian. The fundamental principle underlying Lagrangian mechanics is the principle of stationary action. The action $S$ is defined as the integral of the Lagrangian over time, and the path taken by a system between two points in its configuration space is the one for which the action is stationary:

\begin{equation}
S = \int_{t_1}^{t_2} L(q, \dot{q}, t) \mathrm{d}t . \tag{1} 
\end{equation}

where $q$ represents the generalized coordinates of the system, $\dot{q}$ is the corresponding generalized velocity and $t$ is time. The equations of motion are then obtained by applying the Euler-Lagrange equation:

\begin{equation}
\frac{\mathrm{d}}{\mathrm{d}t}\left(\frac{\partial L}{\partial \dot{q}}\right) - \frac{\partial L}{\partial q} = 0 . \tag{2} 
\end{equation}

This equation is derived from the principle of stationary action and provides a concise expression for the dynamics of the system.
The Lagrangian, denoted by $L$, is defined as the difference between the kinetic and potential energies of a system:

\begin{equation}
L=T-V .\tag{3} 
\end{equation}

To specify the kinetic energies $T$ and potential energies $V$ for this system, we need to define the generalized coordinates. Let $\theta_1$ and $\theta_2$ represent the angles of the first and second pendulum joints, respectively. The corresponding generalized coordinates are $q_1 = \theta_1$ and $q_2 = \theta_2$.



In the following, the ODEs of the double pendulum are to be determined. The derivation of the ODEs is carried out step by step, with tests being used to verify the equations determined. 

### **a)** Determining the potential energy of the system - (1 point)

In this system, $V$ is the sum of the potential energies of the two poles.

**Hint 1:** Potential energy measured at the center of gravity $s$ of a body with a mass $m$:
$$
\begin{align}
V = gms
\end{align}
$$

**Hint 2:** Note the geometric arrangement of the system. In the upper equilibrium, the system has maximum potential energy; in the lower equilibrium, the system has negative potential energy.

In [2]:
function V(q)
    """
    Computes the potential energy of a two-link pendulum system.

    Args:
        q: Vector of joint angles for the two-link pendulum, shape (2).

    Returns:
        Float64: Potential energy of the system.
    """
    g = 9.81
    m_1 = 1
    m_2 = 1
    l_1 = 2
    l_2 = 2

    @assert size(q) == (2,)
    #--- YOUR CODE STARTS HERE ---#
    theta_1, theta_2 = q[1], q[2]
    
    x1 = 0.5 * l_1 * sin(theta_1)
    y1 = -0.5 * l_1 * cos(theta_1)

    x2 = l_2 * (sin(theta_1) + 0.5 * sin(theta_2))
    y2 = -l_2 * (cos(theta_1) + 0.5 * cos(theta_2))

    V = g * m_1 * y1 + g * m_2 * y2
    return V
    #--- YOUR CODE ENDS HERE ---#
end

V (generic function with 1 method)

In [3]:
q = [0.5, 3.0]

println("V: $(V(q))")

@assert isapprox(V(q), -16.1154284047833, atol=1e-5, rtol=0) # evaluation at a single point as help for students for hard check 

V: -16.1154284047833


In [4]:
# Please leave this cell as its is

### **b)** Determining the kinetic energy of the system - (1 point)

The kinetic energy $T$ of the system is the sum of the translational and rotational energies of the bodies.

**Hint 1:** Kinetic energy of a single body measured at its center of gravity $s = [x_s\;y_s]$:
$$
\begin{align}
T = \frac{1}{2}m|v_s|^2 + \frac{1}{2}J\omega^2,
\end{align}
$$
with $v_s = \frac{\mathrm{d}}{\mathrm{d}t}[x_s\; y_s]^\top$ and $\omega = \frac{\mathrm{d}}{\mathrm{d}t}\theta$.

**Hint 2:** Calculate $v_s$ first and, therefore, represent $[x_s\; y_s]^\top$ by the joints' angles. 

**Hint 3:** Inertia of a pole with mass $m$ and length $l$: $J_{\mathrm{pole}} =\frac{1}{12}ml^2$.

In [5]:
function T(q, q_p)
    """
    T(q, q_p)

    Computes the kinetic energy of a two-link pendulum system.

    Args:
        q: Vector of joint angles for the two-link pendulum, shape (2).
        q_p: Vector of joint angular velocities, shape (2).

    Returns:
        float64: Kinetic energy of the system.
    """
    g = 9.81
    m_1 = 1
    m_2 = 1
    l_1 = 2
    l_2 = 2

    @assert size(q) == (2,)
    @assert size(q_p) == (2,)

    #--- YOUR CODE STARTS HERE ---#
    theta_1, theta_2 = q[1], q[2]
    theta_1_dot, theta_2_dot = q_p[1], q_p[2]

    v1_sq = 0.25 * l_1^2 * theta_1_dot^2 
    v2_sq = l_2^2 * (theta_1_dot^2 + 0.25 * theta_2_dot^2 + theta_1_dot * theta_2_dot * cos(theta_1 - theta_2))

    J_1 = 1/12 * m_1 * l_1^2
    J_2 = 1/12 * m_2 * l_2^2

    T = 0.5 * m_1 * v1_sq + 0.5 * m_2 * v2_sq + 0.5 * J_1 * theta_1_dot^2 + 0.5 * J_2 * theta_2_dot^2
    return T
    #--- YOUR CODE ENDS HERE ---#
end

T (generic function with 1 method)

In [6]:
q = [0.5, 3.0]
q_p = [0.4, -2]

println("T: $(T(q, q_p))")

@assert isapprox(T(q, q_p), 4.375163118208428, atol=1e-5, rtol=0) # evaluation at a single point as help for students for hard check 

T: 4.375163118208428


In [7]:
# Please leave this cell as its is

Inserting eq. (3) into eq. (2) delivers:

\begin{equation}
\frac{\partial}{\partial \dot{q}}\left(\frac{\partial T}{\partial \dot{q}}\right)^\top \ddot{q} - \left( -\frac{\partial}{\partial q} \left(\frac{\partial T}{\partial \dot{q}}\right)^\top \dot{q} - \frac{\partial}{\partial t} \left(\frac{\partial T}{\partial \dot{q}}\right)^\top + \frac{\partial T}{\partial q}^\top - \left(\frac{\partial V}{\partial q}\right)^\top \right) = 0. \tag{4}
\end{equation}

We define

\begin{equation}
M(t,q) = \frac{\partial}{\partial \dot{q}}\left(\frac{\partial T}{\partial \dot{q}}\right)^\top
\end{equation}

as the mass matrix and

\begin{equation}
h(t,q,\dot{q}) =  \left( -\frac{\partial}{\partial q} \left(\frac{\partial T}{\partial \dot{q}}\right)^\top \dot{q} - \frac{\partial}{\partial t} \left(\frac{\partial T}{\partial \dot{q}}\right)^\top + \frac{\partial T}{\partial q}^\top - \left(\frac{\partial V}{\partial q}\right)^\top \right)
\end{equation}

the h-vector, which summarizes all gyroscopic and active forces.
We can therefore write the equation again in simplified form:

\begin{equation}
M(t,q) \ddot{q} - h(t,q,\dot{q}) =  0 . \tag{5}
\end{equation}

### **c)** Calculation $M(t,q)$ and $h(t,q,\dot{q})$ - (2 points)

**Hint:** Here you should calculate the derivatives by hand based on your results for $T$ and $V$ from the previous subtasks.

In [8]:
function M(q, m_1, m_2, l_1, l_2)
    """
    Computes the mass matrix for a two-link pendulum system.

    Args:
        q: Vector of joint angles for the two-link pendulum, shape (2).
        m_1: Mass of the first pendulum link.
        m_2: Mass of the second pendulum link.
        l_1: Length of the first pendulum link.
        l_2: Length of the second pendulum link.

    Returns:
        Matrix: Mass matrix of the system, shape (2, 2).
    """
    g = 9.81

    @assert size(q) == (2,)
    #--- YOUR CODE STARTS HERE ---#
    J_1 = 1/12 * m_1 * l_1^2
    J_2 = 1/12 * m_2 * l_2^2

    theta_1, theta_2 = q[1], q[2]

    m11 = 0.25 * m_1 * l_1^2 + m_2 * l_2^2 + J_1
    m12 = 0.5 * m_2 * l_2^2 * cos(theta_1 - theta_2)
    m21 = 0.5 * m_2 * l_2^2 * cos(theta_1 - theta_2)
    m22 = 0.25 * m_2 * l_2^2 + J_2

    mass_matrix = [m11 m12; m21 m22]
    #--- YOUR CODE ENDS HERE ---#

    @assert size(mass_matrix) == (2, 2)
    return mass_matrix
end

M (generic function with 1 method)

In [9]:
q = [0.5, 3.0]


m_1 = 1
m_2 = 1
l_1 = 2
l_2 = 2

println("M: $(M(q, m_1, m_2, l_1, l_2))")

@assert isapprox(M(q, m_1, m_2, l_1, l_2), [5.333333333333333 -1.6022872310938674; -1.6022872310938674 1.3333333333333333], atol=1e-5, rtol=0) # evaluation at a single point as help for students for hard check 

M: [5.333333333333333 -1.6022872310938674; -1.6022872310938674 1.3333333333333333]


In [10]:
# Please leave this cell as its is

In [11]:
# Please leave this cell as its is

In [12]:
function h(q, q_p, m_1, m_2, l_1, l_2)
    """
    Computes the h-vector for a two-link pendulum system.

    Args:
        q: Vector of joint angles for the two-link pendulum, shape (2).
        q_p: Vector of joint angular velocities, shape (2).
        m_1: Mass of the first pendulum link.
        m_2: Mass of the second pendulum link.
        l_1: Length of the first pendulum link.
        l_2: Length of the second pendulum link.

    Returns:
        Vector: h-vector of the system, shape (2).
    """
    g = 9.81

    @assert size(q) == (2,)
    @assert size(q_p) == (2,)

    #--- YOUR CODE STARTS HERE ---#
    theta_1, theta_2 = q[1], q[2]
    theta_1_dot, theta_2_dot = q_p[1], q_p[2]

    d11 = -0.5 * m_2 * l_2^2 * theta_2_dot * sin(theta_1 - theta_2)
    d12 = 0.5 * m_2 * l_2^2 * theta_2_dot * sin(theta_1 - theta_2)
    d21 = -0.5 * m_2 * l_2^2 * theta_1_dot * sin(theta_1 - theta_2)
    d22 = 0.5 * m_2 * l_2^2 * theta_1_dot * sin(theta_1 - theta_2)

    first = -[d11 d12; d21 d22] * q_p

    s1 = -0.5 * m_2 * l_2^2 * theta_1_dot * theta_2_dot * sin(theta_1 - theta_2)
    s2 = 0.5 * m_2 * l_2^2 * theta_1_dot * theta_2_dot * sin(theta_1 - theta_2)

    second = [s1, s2]
    
    t1 = 0.5 * g * m_1 * l_1 * sin(theta_1) + g * m_2 * l_2 * sin(theta_1)
    t2 = 0.5 * g * m_2 * l_2 * sin(theta_2)

    third = [t1, t2]

    h_vector = first .+ second .- third
    #--- YOUR CODE ENDS HERE ---#

    @assert size(h_vector) == (2,)
    return h_vector
end

h (generic function with 1 method)

In [13]:
q = [0.5, 3.0]
q_p = [-1.0, 0.2]

m_1 = 1
m_2 = 1
l_1 = 2
l_2 = 2

println("h: $(h(q,q_p, m_1, m_2, l_1, l_2))")

@assert isapprox(h(q, q_p, m_1, m_2, l_1, l_2), [-14.061615829593379, -2.5813315672752104], atol=1e-5, rtol=0) # evaluation at a single point as help for students for hard check 

h: [-14.061615829593379, -2.5813315672752104]


In [14]:
# Please leave this cell as it is

In [15]:
# Please leave this cell as it is

By rearranging eq. (5), we obtain

\begin{equation}
\ddot{q} =  M(t,q)^{-1}h(t,q,\dot{q}) \tag{6},
\end{equation}

which is now to be converted into the state-space form in the last step.
The following state vector is introduced for this purpose:

\begin{equation}
x =  [\theta_1\;\theta_2\;\dot{\theta_1}\;\dot{\theta_2}]^\top.
\end{equation}

Which in turn results in the following state-space equation:

\begin{equation}
\dot{x} =  [\dot{q}\;\ddot{q}]^\top = [\dot{q}\;M(t,q)^{-1}h(t,q,\dot{q})]^\top = f(x,t).
\end{equation}

### **d)** Convert the equation of motion to the state-space representation - (1 point)

In [16]:
function double_pendulum!(dx, x, p, t)
    """
    Computes the derivatives of the state vector `x` for a double pendulum system.

    Args:
        dx: Vector to store the computed derivatives.
        x: State vector containing the generalized coordinates and their derivatives.
            - x[1]: Angle Θ₁ (first pendulum)
            - x[2]: Angle Θ₂ (second pendulum)
            - x[3]: Angular velocity ω₁ (first pendulum)
            - x[4]: Angular velocity ω₂ (second pendulum)
        p: Parameters vector containing the lengths of the pendulum arms.
            - p[1]: Length of the first pendulum arm (l₁)
            - p[2]: Length of the second pendulum arm (l₂)
        t: Current time (not used in the function).

    Returns:
        Vector: Computed derivatives of the state vector `x`.
            - dx[1]: Angular velocity ω₁ (first pendulum)
            - dx[2]: Angular velocity ω₂ (second pendulum)
            - dx[3]: Angular acceleration ω₁' (first pendulum)
            - dx[4]: Angular acceleration ω₂' (second pendulum)
    """

    g = 9.81
    m_1 = 1
    m_2 = 0.5
    l_1, l_2 = p

    @assert size(x) == (4,)
    @assert length(p) == 2

    # generalized coordinates
    q = x[1:2]
    q_p = x[3:4]

    #--- YOUR CODE STARTS HERE ---#
    q_pp = inv(M(q, m_1, m_2, l_1, l_2)) * h(q, q_p, m_1, m_2, l_1, l_2)

    dx[1] = q_p[1]
    dx[2] = q_p[2]
    dx[3] = q_pp[1]
    dx[4] = q_pp[2]
    #--- YOUR CODE ENDS HERE ---#
end

double_pendulum! (generic function with 1 method)

In [17]:
x0 = [π / 2, π / 2, 0, 0]
p = [1, 1]
tspan = [0, 0.1]
dt = 1e-3
prob = ODEProblem(double_pendulum!, x0, tspan, p);
sol = solve(prob, Tsit5(), saveat=dt);

t, x = sol.t, sol.u
x = reduce(hcat, x)

@assert isapprox(x[:, end], [1.5039951673202, 1.5973123079743048, -1.3326403676290886, 0.5208390338127351], atol=1e-3, rtol=0) # evaluation at a single point as help for students for hard check

In [18]:
# Please leave this cell as its is

In [19]:
# Please leave this cell as its is

First, we generate our ground truth data, for example from a test bench of the system to be idendified. We assume that there only additive, uncorrelated and normally distributed measurement noise.

In [20]:
function gen_measurement(ode_function, x0, tspan, dt, σ)
    """
    Generates measurement data for a simulated system trajectory.

    Args:
        ode_function: Function representing the ordinary differential equations (ODEs) of the system dynamics.
        x0: Initial state vector of the system, shape (n_states).
        tspan: Tuple (t_start, t_end) specifying the time frame for simulation.
        dt: Time step for saving the simulation results.
        σ: Standard deviation of the measurement noise.

    Returns:
        Tuple: The resulting trajectory `x` and the noisy measurements `y`.
        - x: State trajectory matrix of size (n_states, n_timepoints).
        - y: Noisy measurements matrix of size (n_measurement_states, n_timepoints).
    """
    p_true = (0.5, 2) # ground truth

    @assert size(x0) == (4,)

    prob = ODEProblem(ode_function, x0, tspan, p_true)
    sol = solve(prob, Tsit5(), saveat=dt)

    t, x = sol.t, sol.u
    x = reduce(hcat, x)

    @assert size(x) == (4, length(t))

    # define and add measurement noise
    R = σ^2 * I(2)
    y = x[1:2, :] + R * randn(2, size(x)[2])

    return (x, y)
end

gen_measurement (generic function with 1 method)

## Task 2 - Implementation of a function that simulates an experiment - (2 points)

In order to estimate the parameters, a function is to be added which carries out a whole simulation with the estimated parameters. Use a standard ODE solver `Tsit5()` to simulate a state trajecory based on an arbitrary initial state.

In [21]:
function sim_exp(ode_function, x0, tspan, dt, p)
    """
    Generates simulation data for a given system and simulation parameters.

    Args:
        ode_function: Function representing the ordinary differential equations (ODEs) of the system dynamics.
        x0: Initial state vector of the system, shape (n_states).
        tspan: Tuple (t_start, t_end) specifying the time frame for simulation.
        dt: Time step for saving the simulation results.
        p: Parameter vector containing system-specific parameters.

    Returns:
        Tuple: The resulting trajectory `x` and the simulated measurements `y`.
        - x: State trajectory matrix of size (n_states, n_timepoints).
        - y: Simulated measurements matrix of size (n_measurement_states, n_timepoints).
    """

    @assert size(x0) == (4,)

    #--- YOUR CODE STARTS HERE ---#
    prob = ODEProblem(ode_function, x0, tspan, p)
    sol = solve(prob, Tsit5(), saveat=dt)
    t, x = sol.t, sol.u
    x = reduce(hcat, x)
    #--- YOUR CODE ENDS HERE ---#

    @assert size(x) == (4, length(t))

    y = x[1:2, :]
    return (x, y)
end

sim_exp (generic function with 1 method)

In [22]:
x0 = [π / 2, π / 2, 0, 0]
tspan = [0, 0.1]
dt = 1e-3
p = [1.0, 1.0]

x_hat, y_hat = sim_exp(double_pendulum!, x0, tspan, dt, p)

@assert size(y_hat) == (2, 101)

println("y_hat: $(y_hat[:,end])")
@assert isapprox(y_hat[:, end], [1.5039951673202, 1.5973123079743048], atol=1e-5, rtol=0)

y_hat: [1.5039951673199368, 1.5973123079750016]


In [23]:
# Please leave this cell as its is

In [24]:
# Please leave this cell as its is

## Task 3: Solve an identification problem for unknown pendulum length values - (1 point)

We assume that we have ideal structural knowledge regarding the double pendulum dynamics, but lack the length information of the two poles. Hence, we want to set up an optimization problem, which identifies the poles' length. First, a cost function must be defined for this purpose.

### **a)** Write a function to compute the MSE. - (0.5 points)

In [25]:
function mse(y, y_hat)
    """
    Computes the Mean Squared Error (MSE) between two matrices or vectors.

    Args:
        y: Matrix or vector of actual values.
        y_hat: Matrix or vector of predicted values.

    Returns:
        Float64: Mean Squared Error between `y` and `y_hat`.
    """
    #--- YOUR CODE STARTS HERE ---#
    diff = sum((y .- y_hat).^2)
    diff /= size(y, 2)

    return diff
    #--- YOUR CODE ENDS HERE ---#
end

mse (generic function with 1 method)

In [26]:
y = ones(2, 5)
y_hat = 3 * ones(2, 5)
l = mse(y, y_hat)

@assert length(l) == 1

println("l: $(l)")
@assert isapprox(l, 8.0, atol=1e-5, rtol=0)

l: 8.0


In [27]:
# Please leave this cell as its is

In [28]:
# Please leave this cell as its is

### **b)** Set up the objective function for the simulation case - (0.5 points)

In [29]:
function simulation_objective(p; y_mes=y_mes, ode_func=double_pendulum!, x0=x0, tspan=tspan, dt=dt)
    """
    Computes the MSE loss value for a simulation-based optimization problem.

    Args:
        p: Parameter vector to be optimized.
        y_mes: Matrix of measured data to compare against simulation results (default: y_mes).
        ode_func: Function representing the ordinary differential equations (ODEs) of the system dynamics (default: double_pendulum!).
        x0: Initial state vector of the system (default: x0).
        tspan: Tuple (t_start, t_end) specifying the time frame for simulation (default: tspan).
        dt: Time step for saving the simulation results (default: dt).

    Returns:
        Float64: Objective value representing the mean squared error between measured and simulated data.
    """
    #--- YOUR CODE STARTS HERE ---#
    x_sim, y_sim = sim_exp(ode_func, x0, tspan, dt, p)
    return mse(y_sim, y_mes)
    #--- YOUR CODE ENDS HERE ---#
end

simulation_objective (generic function with 1 method)

In [30]:
# Please leave this cell as its is

In [31]:
# Please leave this cell as its is

Based on identification framework, we utilize the Optim.jl package to solve for the pendulum length. This is provided as a executation-ready code, which you can play around with in the following. As we have generated the ground truth data using the above simulation ourselves, you can compare the found results for the length values and check if you have implemented everything correctly. This last step of configuring and executing the optimization is not graded, but serves as a baseline for the next task. 

In [32]:
p_init = [0.7, 1.7] # initial guess of length values
x0 = [π / 2, 0, 0, 0]
tspan = (0, 2)
dt = 1e-3

σ = 0.1 # measurement noise config
solver = GradientDescent() # you can also define other solvers
options = Optim.Options(x_tol=1e-5)

_, y_mes = gen_measurement(double_pendulum!, x0, tspan, dt, σ)

result = optimize(p -> simulation_objective(p, y_mes=y_mes, ode_func=double_pendulum!, x0=x0, tspan=tspan, dt=dt), p_init, solver, options)

# Print the result
print(result)

# print optimal parameters
optimal_parameters = Optim.minimizer(result)
println("Optimal Parameters: ", optimal_parameters)

@assert size(optimal_parameters) == (2,)

 * Status: success

 * Candidate solution
    Final objective value:     1.605721e-01

 * Found with
    Algorithm:     Gradient Descent

 * Convergence measures
    |x - x'|               = 7.14e-11 ≤ 1.0e-05
    |x - x'|/|x'|          = 3.17e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 9.16e-08 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.71e-07 ≰ 0.0e+00
    |g(x)|                 = 1.02e-01 ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    11
    f(x) calls:    114
    ∇f(x) calls:   114
Optimal Parameters: [1.8360447062700427, 2.2524879011357886]


**Hint:** Try changing the variance of the measurement noise and also try other solvers. Gradient descent works well for convex problems, but if the cost landscape is very nonlinear, it needs a good initialization, therefore global algorithms are better suited for problems where you don't have a good initial guess. The `Optim.jl` package comes with various solvers which you can find [here](https://julianlsolvers.github.io/Optim.jl/stable/#).

## Task 4: Solve an identification task on testbench data - (2 points)

In the previous part of the task, we generated the simulation data and presented the toolchain that can be used to solve the problem. In this subtask, measurement data from a double pendulum test rig is now read in so that the true length values are unknown to you in this subtask. Use the existing toolchain to estimate the pole lengths of the pendulum. Your specific solution path is not set in stone as we will only check for the found pendulum length values, which should deviate less than 5 % from the (unknown) ground truth values.

**Hint #1:** The search area can be limited to a range of $[0.1, 3.0]$ for both parameters.

**Hint #2:** You can compare the final loss value after training with the successful identification from the above subtasks to get an idea if the found results in task 4) are feasible or not.

In [33]:
data_1 = matopen("data_task_4.mat");
x_mes = read(data_1, "x");
y_mes = read(data_1, "y");

In [ ]:
p_init = NaN
tspan = (0, 1)
dt = 1e-3
x0 = NaN
solver = NaN
optimal_parameters = NaN

#--- YOUR CODE STARTS HERE ---#
p_init = [1.0, 1.0] # initial guess of length values
x0 = [π / 2, 0, 0, 0]
σ = 0.1 # measurement noise config
solver = GradientDescent() 
options = Optim.Options(x_tol=1e-5)
result = optimize(p -> simulation_objective(p, y_mes=y_mes, ode_func=double_pendulum!, x0=x0, tspan=tspan, dt=dt), p_init, solver, options)

# Print the result
print(result)

# print optimal parameters
optimal_parameters = Optim.minimizer(result)

@assert size(optimal_parameters) == (2,)
#--- YOUR CODE ENDS HERE ---#

println("Optimal Parameters: ", optimal_parameters)

 * Status: success

 * Candidate solution
    Final objective value:     4.127437e+00

 * Found with
    Algorithm:     Gradient Descent

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 1.0e-05
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.98e-04 ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    71
    f(x) calls:    226
    ∇f(x) calls:   226
Optimal Parameters: [3.65089492860048, 2.841194432503199]


In [35]:
@assert size(optimal_parameters) == (2,)
